# Transformacion de datos utilizando PySpark

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
path = '/FileStore/tables'

# Lectura de un archivo CSV local
df = spark.read.csv(path + '/data-1.csv', header=True, inferSchema=True)

In [0]:
df.columns

Out[3]: ['Row ID',
 'Order ID',
 'Order Date',
 'Ship Date',
 'Ship Mode',
 'Customer ID',
 'Customer Name',
 'Segment',
 'Country',
 'City',
 'State',
 'Postal Code',
 'Region',
 'Product ID',
 'Category',
 'Sub-Category',
 'Product Name',
 'Sales',
 'Quantity',
 'Discount',
 'Profit']

## Selección de columnas

In [0]:
df.select("Quantity", "Discount").show(5)

+--------+--------+
|Quantity|Discount|
+--------+--------+
|       2|       0|
|       3|       0|
|       2|       0|
|       5|    0.45|
|       2|     0.2|
+--------+--------+
only showing top 5 rows



In [0]:
df.selectExpr("Quantity", "Discount + 1 as Discountmas1").show(5)

+--------+------------+
|Quantity|Discountmas1|
+--------+------------+
|       2|         1.0|
|       3|         1.0|
|       2|         1.0|
|       5|        1.45|
|       2|         1.2|
+--------+------------+
only showing top 5 rows



In [0]:
df_col = df.select("Quantity", "Discount")

## Filtrado de filas

In [0]:
df_col.filter(df.Quantity > 2).show(5)

+--------+--------+
|Quantity|Discount|
+--------+--------+
|       3|       0|
|       5|    0.45|
|       7|       0|
|       4|       0|
|       6|     0.2|
+--------+--------+
only showing top 5 rows



In [0]:
df.where(df.col1.isNull()).show(5)

## Renombrar columnas

In [0]:
df_col = df_col.withColumnRenamed("Quantity", "Cantidades")
df_col.show(5)

+----------+--------+
|Cantidades|Discount|
+----------+--------+
|         2|       0|
|         3|       0|
|         2|       0|
|         5|    0.45|
|         2|     0.2|
+----------+--------+
only showing top 5 rows



## Cálculos y generación de columnas

In [0]:
df_col = df_col.withColumn("new_column", df_col.Cantidades + df_col.Discount)
df_col.show(5)

+----------+--------+----------+
|Cantidades|Discount|new_column|
+----------+--------+----------+
|         2|       0|       2.0|
|         3|       0|       3.0|
|         2|       0|       2.0|
|         5|    0.45|      5.45|
|         2|     0.2|       2.2|
+----------+--------+----------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import when

df.withColumn("new_column", when(df.col1 > 5, "High").otherwise("Low"))

In [0]:
from pyspark.sql.functions import expr

df.withColumn("new_column", expr("CASE WHEN col1 > 5 THEN 'High' ELSE 'Low' END"))

## Fusionar DataFrames

In [0]:
df1.join(df2, df1.col1 == df2.col1, "inner")

In [0]:
df1.union(df2)

## Obtener elementos únicos

In [0]:
df.distinct()

## Agrupacion y ordenamiento

In [0]:
df.orderBy("col1")

In [0]:
df.groupBy("col1").agg({"col2": "max"})

In [0]:
df.orderBy("col1").first()

## Reemplazar caracteres

In [0]:
from pyspark.sql.functions import regexp_replace

df.withColumn("new_column", regexp_replace('col', 'actual', 'nuevo'))

## windows functions

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, max

# Definir la ventana de particionamiento
windowSpec = Window.partitionBy("producto")

# Agregar la columna "max_precio" con el valor máximo, particionado por "producto"
df.withColumn("max_precio", max(col("precio")).over(windowSpec))
